"""
This script is used to plot the Global mean surface air temperature (GMSAT) from observation and multimodel simulation.
"""

## import observation data

In [ ]:
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from pathlib import Path
import glob
# %%
# define function
import src.SAT_function_Obs_Fingerprint as data_process
import src.Data_Preprocess as preprocess

In [ ]:
# subplot a: global mean temperature anomalies during 1850-2022 HadCRUT5
input_observation = './Supp_Figure1/'

tas_HadCRUT_annual = xr.open_dataset(input_observation + 'tas_HadCRUT5_annual_anomalies.nc')

In [ ]:
tas_HadCRUT_annual

In [ ]:
# tas_HadCRUT_annual = tas_HadCRUT_annual.rename({'__xarray_dataarray_variable__': 'tas_HadCRUT5'})

In [ ]:
tas_HadCRUT_annual_1850_2022 = tas_HadCRUT_annual.sel(year=slice('1993', '2022')).tas

In [ ]:
tas_HadCRUT5_annual_ano = tas_HadCRUT_annual_1850_2022.mean(dim=['year'])

In [ ]:
tas_HadCRUT5_annual_ano.min().values, tas_HadCRUT5_annual_ano.max().values

## Input the multimodel mean GMST data

In [ ]:
# load the data
# MMEM GMSAT annual regression coefficient
dir_path = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Figure1/'

beta_MMEM_HadCRUT5_annual = xr.open_dataset(dir_path + 'HadCRUT_slope_Beta_coefficients_MMEM_TS.nc')
alpha_MMEM_HadCRUT5_annual = xr.open_dataset(dir_path + 'HadCRUT_intercept_Alpha_constant_MMEM_TS.nc')

In [ ]:
beta_MMEM_HadCRUT5_annual = beta_MMEM_HadCRUT5_annual.rename({'__xarray_dataarray_variable__': 'beta_MMEM_HadCRUT5'})
alpha_MMEM_HadCRUT5_annual = alpha_MMEM_HadCRUT5_annual.rename({'__xarray_dataarray_variable__': 'alpha_MMEM_HadCRUT5'})

In [ ]:
beta_MMEM_HadCRUT5_annual['beta_MMEM_HadCRUT5'].min().values, beta_MMEM_HadCRUT5_annual['beta_MMEM_HadCRUT5'].max().values

In [ ]:
alpha_MMEM_HadCRUT5_annual['alpha_MMEM_HadCRUT5'].min().values, alpha_MMEM_HadCRUT5_annual['alpha_MMEM_HadCRUT5'].max().values

### Subplot d: the reconstructed y mean and the y residual from the raw data

In [ ]:
dir_const = '/work/mh0033/m301036/Land_surf_temp/Disentangling_OBS_SAT_trend/Supp_Figure1/data/'

GSAT_estimate_forced = xr.open_dataset(dir_const + 'GSAT_HadCRUT5_Forced_anomalies_1850_2022.nc')
GSAT_estimate_internal = xr.open_dataset(dir_const + 'GSAT_HadCRUT5_Internal_Variability_anomalies_1850_2022.nc')

In [ ]:
GSAT_estimate_forced = GSAT_estimate_forced.rename({'__xarray_dataarray_variable__': 'GSAT_HadCRUT5_Forced'})
GSAT_estimate_internal = GSAT_estimate_internal.rename({'__xarray_dataarray_variable__': 'GSAT_HadCRUT5_Internal'})

In [ ]:
# Calculate the last 30years mean
GSAT_estimate_forced_1993_2022 = GSAT_estimate_forced.sel(year=slice('1993', '2022')).GSAT_HadCRUT5_Forced

In [ ]:
GSAT_estimate_forced_mean = GSAT_estimate_forced_1993_2022.mean(dim=['year'])
# GSAT_estimate_internal_mean = GSAT_estimate_internal['GSAT_HadCRUT5_Internal'].mean(dim=['year'])

In [ ]:
GSAT_estimate_forced_mean.min().values, GSAT_estimate_forced_mean.max().values

In [ ]:
GSAT_estimate_internal_1993_2022 = GSAT_estimate_internal.sel(year=slice('1993', '2022')).GSAT_HadCRUT5_Internal
GSAT_estimate_internal_mean = GSAT_estimate_internal_1993_2022.mean(dim=['year'])
GSAT_estimate_internal_mean.min().values, GSAT_estimate_internal_mean.max().values

## Plotting 

In [ ]:
import svgwrite
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerLine2D
from matplotlib.legend import Legend
import matplotlib.lines as Line2D

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import seaborn as sns
from matplotlib.colors import ListedColormap
from matplotlib.colors import BoundaryNorm, ListedColormap

In [ ]:
#Plotting
# setting the parameters for the figure
plt.rcParams['figure.figsize'] = (8, 10)
plt.rcParams['font.size'] = 16
# plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['ytick.direction'] = 'out'
# plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['ytick.major.right'] = True
plt.rcParams['ytick.right'] = True
plt.rcParams['xtick.bottom'] = True
plt.rcParams['savefig.transparent'] = True # save the figure with a transparent background

In [ ]:
# Setting the range of the x-axis
x = np.arange(1850, 2023, 1)

In [ ]:
obs_name = ["HadCRUT5"]
model_names = ["CanESM5(50)", "IPSL-CM6A-LR(32)", "EC-Earth3(21)", "ACCESS-ESM1.5(40)", "MPI-ESM1.2-LR(50)","MIROC6(50)"]

RGB_dict = {'CanESM5(50)':np.array([50, 34, 136])/255., 
            'IPSL-CM6A-LR(32)':np.array([68, 170, 152])/255., 
            'EC-Earth3(21)':np.array([221, 204, 118])/255., 
            'ACCESS-ESM1.5(40)':np.array([204, 101, 119])/255.,
            'MPI-ESM1.2-LR(50)':np.array([170, 67, 153])/255., 
            'MIROC6(50)':np.array([136, 33, 85])/255., 
            'MME':np.array([0, 0, 0])/255.}

In [ ]:
c_obs = "orange"
c_obs_1 = "blue"
c_obs_2 = "green" #'darkred'

obs_color = {
    "MLOST": c_obs,
    "HadCRUT5": c_obs_1,
    "Berkeley": c_obs_2
}

for obs in obs_name:
    print(obs)
    obs_color[obs]
    print(obs_color[obs])
    
lw_obs = 1.5
lw_model = 2.5
xmin, xmax = 1850, 2022
ymin, ymax = -1.5, 2.0

In [ ]:
RGB_dict[model_names[0]],model_names[0]

In [ ]:
# define the contourf plot function
def plot_data(data,lats,lons,levels=None, extend=None,cmap=None, title="", ax=None, show_xticks=False, show_yticks=False):
    if ax is None:
        fig, ax = plt.subplots(subplot_kw={'projection': ccrs.Robinson()})
    
    # ax.set_extent([lons.min(), lons.max(), lats.min(), lats.max()], crs=ccrs.PlateCarree())
    # Add coastlines
    ax.coastlines(resolution='110m')
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,
                      colors='gray', alpha=0.5, linestyle='--', linewidth=0.5)

    # Disable labels on the top and right of the plot
    gl.top_labels = False
    gl.right_labels = False

    # Enable labels on the bottom and left of the plot
    gl.bottom_labels = show_xticks
    gl.left_labels = show_yticks
    gl.xformatter = cticker.LongitudeFormatter()
    gl.yformatter = cticker.LatitudeFormatter()
    gl.xlabel_style = {'size': 14}
    gl.ylabel_style = {'size': 14}
    
    if show_xticks:
        gl.bottom_labels = True
    if show_yticks:
        gl.left_labels = True
    # Add filled contour plot
    cf = ax.contourf(lons, lats, data, levels=levels, extend=extend, cmap=cmap, transform=ccrs.PlateCarree())
    
    # Add title
    ax.set_title(title, loc='center', fontsize=22, pad=5.0)
    return cf

In [ ]:
import matplotlib.pyplot as plt
import patchworklib as pw
import cartopy.util as cutil


In [ ]:
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.colors import BoundaryNorm
import cartopy.util as cutil
import seaborn as sns
import matplotlib.colors as mcolors
import palettable
#  cmap = mcolors.ListedColormap(palettable.scientific.diverging.Vik_20.mpl_colors)
cmap=mcolors.ListedColormap(palettable.cmocean.diverging.Balance_20.mpl_colors)

In [ ]:
# Four subplots
"""
    subplot a: global mean temperature anomalies during 1850-2022 HadCRUT5; contourf plot
    subplot b: model simulated GSAT timeseries in each SMILEs and MMEM 1850-2022; x-y line plot
    subplot c: GSAT anomalies Beta and Alpha coefficients; contourf plot (from top to bottom: Beta, Alpha two subplots concatenated)
    subplot d: GSAT anomalies spatial pattern by construction, with the internal variability and forced component; contourf plot 
    (from top to bottom: forced component, internal variability two subplots concatenated)
"""

### Plot the third schematic diagram of the linear regression model to 
### extract the forced pattern with the alpha and betha scaling factors and the constant stands for intercepts

In [ ]:
# Create the figure and a GridSpec layout
fig = plt.figure(figsize=(18, 10))
gs = gridspec.GridSpec(
    2, 3,
    height_ratios=[1, 1],
    width_ratios=[1, 1, 1],  # make col 0 narrower to center a+b better
    hspace=0.15, wspace=0.1
)

# ===== Row 1: Two panels centered =====
ax0 = fig.add_subplot(gs[0, 0], projection=ccrs.Robinson(180))  # left (beta)
ax1 = fig.add_subplot(gs[0, 1], projection=ccrs.Robinson(180))  # right (alpha)

# ===== Row 2: Three panels full width =====
ax2 = fig.add_subplot(gs[1, 0], projection=ccrs.Robinson(180))  # y
ax3 = fig.add_subplot(gs[1, 1], projection=ccrs.Robinson(180))  # y_hat
ax4 = fig.add_subplot(gs[1, 2], projection=ccrs.Robinson(180))  # residuals

# === Plot 1: Beta and Alpha ===
bounds = np.arange(-1.0, 1.1, 0.1)
bounds_alpha = np.arange(-0.5, 0.55, 0.05)

beta_anomalies = beta_MMEM_HadCRUT5_annual['beta_MMEM_HadCRUT5']
beta_anomalies_with_cyclic, lon_cyclic = cutil.add_cyclic_point(beta_anomalies, coord=beta_MMEM_HadCRUT5_annual['lon'])
contour_beta = plot_data(beta_anomalies_with_cyclic, beta_MMEM_HadCRUT5_annual['lat'], lon_cyclic, levels=bounds,
                         extend='both', cmap='RdBu_r', title="Regression Coefficient ("+r"$\beta$"+")", ax=ax0,
                         show_xticks=True, show_yticks=True)

alpha_anomalies = alpha_MMEM_HadCRUT5_annual['alpha_MMEM_HadCRUT5']
alpha_anomalies_with_cyclic, lon_cyclic = cutil.add_cyclic_point(alpha_anomalies, coord=alpha_MMEM_HadCRUT5_annual['lon'])
contour_alpha = plot_data(alpha_anomalies_with_cyclic, alpha_MMEM_HadCRUT5_annual['lat'], lon_cyclic, levels=bounds_alpha,
                          extend='both', cmap='RdBu_r', title="Intercept ("+r"$\alpha$"+")", ax=ax1,
                          show_xticks=True, show_yticks=False)

# === Plot 2: y, y_hat, residuals ===
level_forced = np.arange(-1.0, 1.1, 0.1)
level_internal = np.arange(-0.5, 0.55, 0.05)

tas_obs_with_cyclic, lon_cyclic = cutil.add_cyclic_point(tas_HadCRUT5_annual_ano, coord=tas_HadCRUT5_annual_ano['lon'])
contour_y = plot_data(tas_obs_with_cyclic, tas_HadCRUT5_annual_ano['lat'], lon_cyclic, levels=level_forced,
                      extend='both', cmap='RdBu_r', title=r'$y$', ax=ax2,
                      show_xticks=True, show_yticks=True)

forced_with_cyclic, lon_cyclic = cutil.add_cyclic_point(GSAT_estimate_forced_mean, coord=GSAT_estimate_forced_mean['lon'])
contour_yhat = plot_data(forced_with_cyclic, GSAT_estimate_forced_mean['lat'], lon_cyclic, levels=level_forced,
                         extend='both', cmap='RdBu_r', title=r'$\hat{y} = \beta \langle \bar{x}_t \rangle + \alpha$', ax=ax3,
                         show_xticks=True, show_yticks=False)

internal_with_cyclic, lon_cyclic = cutil.add_cyclic_point(GSAT_estimate_internal_mean, coord=GSAT_estimate_internal_mean['lon'])
contour_resid = plot_data(internal_with_cyclic, GSAT_estimate_internal_mean['lat'], lon_cyclic, levels=level_internal,
                          extend='both', cmap='RdBu_r', title=r'$\epsilon = y - \hat{y}$', ax=ax4,
                          show_xticks=True, show_yticks=False)

# === Colorbars ===
cbar_ax1 = fig.add_axes([0.135, 0.52, 0.22, 0.02])  # For beta
cbar1 = plt.colorbar(contour_beta, cax=cbar_ax1, orientation='horizontal')
cbar1.ax.tick_params(labelsize=14)

cbar_ax2 = fig.add_axes([0.4, 0.52, 0.22, 0.02])  # For alpha
cbar2 = plt.colorbar(contour_alpha, cax=cbar_ax2, orientation='horizontal')
cbar2.ax.tick_params(labelsize=14)

cbar_ax3 = fig.add_axes([0.2, 0.12, 0.35, 0.02])  # For y/yhat
cbar3 = plt.colorbar(contour_y, cax=cbar_ax3, orientation='horizontal')
cbar3.set_label('SAT anomalies (°C)', fontsize=16, labelpad=5)
cbar3.ax.tick_params(labelsize=14)

cbar_ax4 = fig.add_axes([0.68, 0.12, 0.22, 0.02])  # For residuals
cbar4 = plt.colorbar(contour_resid, cax=cbar_ax4, orientation='horizontal')
cbar4.set_label('SAT anomalies (°C)', fontsize=16, labelpad=5)
cbar4.ax.tick_params(labelsize=14)

# === Subplot labels ===
ax0.text(-0.05, 1.05, 'a', transform=ax0.transAxes, fontsize=20, fontweight='bold')
ax1.text(-0.05, 1.05, 'b', transform=ax1.transAxes, fontsize=20, fontweight='bold')
ax2.text(-0.05, 1.05, 'c', transform=ax2.transAxes, fontsize=20, fontweight='bold')
ax3.text(-0.05, 1.05, 'd', transform=ax3.transAxes, fontsize=20, fontweight='bold')
ax4.text(-0.05, 1.05, 'e', transform=ax4.transAxes, fontsize=20, fontweight='bold')

# === Save figure ===
fig.savefig("Extended_Fig1_combined.png", dpi=300, bbox_inches='tight')
fig.savefig("Extended_Fig1_combined.pdf", format='pdf', dpi=300, bbox_inches='tight')
fig.savefig("Extended_Fig1_combined.eps", format='eps', dpi=300, bbox_inches='tight')
plt.show()